# 可计算文档MVP说明
`lixizhi, 2023.5.23`
- 对标 jupyter或 google codelabs
- 针对计算机教育中的数字教材与测评的应用场景
- 基于国产云Docker + Keepwork + Web Paracraft 

## MVP 开发计划
- 1. 技术预研：jupyter 及其生态研究说明（包括jupyter 交互协议及生态组建）【本周内】
- 2. 交互计算中间件github项目启动【5月31号】
- 3. keepwork支持可计算文档构建【6月30号】
- 4. 与课程平台结合、同时与出版社发布可计算文档教材（关于深度学习的一节样课）【7月31号】
- 5. 形成可交付方案（产品和课程）【8月31号】

## Keepwork研发内容
keepwork editor中增加新的ipynb的文档类型和对应的编辑器, 直接用jupyter的editor即可, 先支持本地的python环境127.0.0.1:8888
在新建文档时，多一个可计算文档的选项，同时原先的md也支持可计算文档，并可导出ipynb. 
参考：
- https://github.com/microsoft/vscode-jupyter/tree/main
- https://github.com/jupyterhub/jupyterhub/tree/main

特色功能：
- 支持 云主机和本地主机自由切换 (MVP需要支持)
- 支持 web paracraft 实验室 (1.0支持)
- 支持 web NPL CAD （2.0支持）
- 支持 图形化代码编程 (2.0支持)

### 特色1：云主机和本地主机自由切换
参考google codelabs,  先支持本地localhost server. 能够演示。 

In [1]:
def TestPython():
   for i in range(5):
      print("Hello World!")

TestPython()

Hello World!
Hello World!
Hello World!
Hello World!
Hello World!



### 特色2：web paracraft 实验室 （ParaLab）
之前很多人不会使用paracraft的代码方块，觉得UI藏得太深了，找不到。 我们采用可计算文档cells的方式来自动创建多个代码方块。注意：
- 一个文档默认只能对应一个paracraft世界 （web paracraft instance）。 
- 每个paracraft文档代码块自动对应3D场景中的1个(自动生成的)代码方块。 
- 与Python的文本输出不一样， 当我们用鼠标滑动文档的位置时，web paracraft窗口也会自动移动到代码块的下方，并自动切换到场景中对应的位置。 
- 我们可以根据代码的上级的`###` 加编号来对代码方块起名字，例如`表与数组1`,代表在这个标题下的第1个cell中的代码。
   - 也可以通过首行注释 `-- title: 代码方块的名字` 强制定义代码方块的名字。
- 默认会创建一个代码方块、一个拉杆和一个告示牌， 告示牌为上级的`###`名字。
- 可以通过首行注释中空格的数量，来决定自动生成的2个代码方块在3D场景中的距离。 
   - 默认方向是Z+， 默认起点为出生点，如果周围不是空地自动找最近的空地。
- 每个文档对应的3D世界以文档的名字命名，如果世界不存在则自动创建空白平坦世界， 世界存在web browser disk cache中。 
- 在文档中，可以用jupyter方式执行代码方块，执行后，拉杆自动激活。但是默认不保存世界。
- 用户可以通过命令的方式分享或保存本地世界，也可以直接在web paracraft输出窗口中（只有一个wasm instance）编辑或存盘。
- 高级用户可以手工改变自动生成的代码方块的位置，因为文档运行时，会自动寻找代码方块，如果已经存在同名的，就不会再次创建了。
   - 高级用户可以通过/loadworld -projectid的方式在文档开始时，自动加载一个云端的世界，类似google codelabs的网盘登录功能（都是用python代码实现的）。 
- 如果代码方块存在log,echo类的输出，则此类输出，需要保存到可计算文档中， 和jupyter的方式一样，作为cell output。

下面是一个例子：

#### 2.6 表与数组
我们之前讲过， NPL语言中的全部数据类型是： 数字，字符串，表，函数，true/false/nil。
我们只剩下表和函数没有讲了。 本节我们讲解：表(Table)。请看下面的例子

In [ ]:
-- 表与数组

-- moveTo(19202, 5, 19168)
-- turnTo(90)

local params = {}
params.pos = {x=19202, y=5, z=19168}
params.facing = 90

moveTo(params.pos.x, params.pos.y, params.pos.z)
turnTo(params.facing)

输出：

![](https://api.keepwork.com/storage/v0/siteFiles/2424/raw#image.png)

#### 图块演示 
上面的3D窗口是采用本地的web paracraft渲染的， 如果文档中有多个， 其实是通过一个实例移动位置完成的，这样可以0秒查看之前的结果。
除了文本编程， 我们在v2.0阶段也可以考虑图形化编程，如下图在文档中直接支持图块编程。
- 注意1： 图块和markdown一样可以手工在编辑模式和预览模式中切换。 预览模式不显示左侧的图块列表。 

![](https://api.keepwork.com/storage/v0/siteFiles/2736/raw#image.png)

输出：

![](https://api.keepwork.com/storage/v0/siteFiles/2734/raw#image.png)


## 开发任务分配

1.  直接在keepwork 的原始markdown编辑器支持新插件`paracraft_codeblock`. 这个作为一个基准版本去测试和使用。 @wxa  6月30日
    - 用emscripten 编译一个Python3运行时环境，内置到web paracraft中, web版支持python代码方块。  @wxa 7月30日
    - Fork VS code plugin:  https://github.com/microsoft/vscode-jupyter  并通过vs code extension 支持paracraft web vm.  @wxa 
1.  一个全新的ipynb文件的keepwork页面编辑器，直接使用开源vscode jupyter改造。 @big  6月30日 
    - ipynb文件支持`paracraft_codeblock` cell类型。 @big  7月15日
    - 支持至少一种jupyter docker容器 @zhaopeiran 7 月30日
1.  产品、前端介入，将keepwork和课程中台打通。 @liyu @zhaopeiran @liufengfeng 7月30日
1.  测试、市场、课程介入，构建课例和可商业化解决方案。 @pengchen @lixizhi 7月30日
1. 可选，nice to have： 图块的支持 @wxa 12月